In [163]:
# %pip install numpy pandas pickle
# %pip install matplotlib

In [164]:
import numpy as np
import pandas as pd
import pickle


class ConvolutionLayer:

    def __init__(self, n_filter, filter_size, stride, padding):
        self.filter_size = filter_size
        self.n_filter = n_filter
        self.stride = stride
        self.padding = padding

        self.filters = None
        self.biases = None

    
    def forward(self, input):        
        batch_size, n_channel, height, width = input.shape
        output_shape = (batch_size, self.n_filter, int((height - self.filter_size + 2*self.padding)/self.stride + 1), int((width - self.filter_size + 2*self.padding)/self.stride + 1))
        output = np.zeros(output_shape)

        if self.filters is None:
            self.filters = np.random.randn(self.n_filter, n_channel, self.filter_size, self.filter_size) / np.sqrt(2 / (self.filter_size * self.filter_size * n_channel))
        if self.biases is None:
            self.biases = np.random.randn(self.n_filter)

        if self.padding > 0:
            input = np.pad(input, ((0,0), (0,0), (self.padding, self.padding), (self.padding, self.padding)), 'constant')
        
        for b in range(batch_size):
            for c in range(self.n_filter):
                for h in range(height):
                    for w in range(width):
                        output[b, c, h, w] = np.sum(input[b, :, h*self.stride :h*self.stride + self.filter_size, w*self.stride : w*self.stride + self.filter_size] * self.filters[c, :, :, :,]) + self.biases[c]

        return output
    

    # def backward(self, output, learning_rate):
    #     # perform back propagation for convolution

    #     batch_size, n_channel, height, width = output.shape
    #     input_shape = (batch_size, n_channel, height, width)
    #     input = np.zeros(input_shape)

    #     if self.padding > 0:
    #         output = np.pad(output, ((0,0), (0,0), (self.padding, self.padding), (self.padding, self.padding)), 'constant')
        
    #     for b in range(batch_size):
    #         for c in range(self.n_filter):
    #             for h in range(height):
    #                 for w in range(width):
    #                     input[b, :, h*self.stride :h*self.stride + self.filter_size, w*self.stride : w*self.stride + self.filter_size] += output[b, c, h, w] * self.filters[c, :, :, :,]
    #                     self.filters[c, :, :, :,] -= learning_rate * output[b, c, h, w] * input[b, :, h*self.stride :h*self.stride + self.filter_size, w*self.stride : w*self.stride + self.filter_size]
    #                     self.biases[c] -= learning_rate * output[b, c, h, w]
        
    #     return input

In [165]:
class ReLUActivationLayer:

    def forward(self, input):
        return np.maximum(input, 0)

    def backward(self, output, learning_rate):
        return np.where(output > 0, 1, 0)

class MaxPoolingLayer:

    def __init__(self, pool_size, stride):
        self.pool_size = pool_size
        self.stride = stride

    def forward(self, input):
        batch_size, n_channel, height, width = input.shape

        output_h = int((height - self.pool_size)/self.stride + 1)
        output_w = int((width  - self.pool_size)/self.stride + 1)

        output_shape = (batch_size, n_channel, output_h, output_w)
        output = np.zeros(output_shape)

        for b in range(batch_size):
            for c in range(n_channel):
                for h in range(output_h):
                    for w in range(output_w):
                        output[b, c, h, w] = np.max(input[b, :, h*self.stride :h*self.stride + self.pool_size, w*self.stride : w*self.stride + self.pool_size])

        return output

    # def backward(self, output, learning_rate):
    #     batch_size, n_channel, height, width = output.shape
    #     input_shape = (batch_size, n_channel, height, width)
    #     input = np.zeros(input_shape)

    #     for b in range(batch_size):
    #         for c in range(n_channel):
    #             for h in range(height):
    #                 for w in range(width):
    #                     input[b, c, h*self.stride :h*self.stride + self.pool_size, w*self.stride : w*self.stride + self.pool_size] = np.where(input[b, c, h*self.stride :h*self.stride + self.pool_size, w*self.stride : w*self.stride + self.pool_size] == np.max(input[b, c, h*self.stride :h*self.stride + self.pool_size, w*self.stride : w*self.stride + self.pool_size]), output[b, c, h, w], 0)

        # return input    
        # 

class SoftMaxLayer:

    def forward(self, input):
        val = np.exp(input) / np.exp(input).sum()
        return val

    def fwd(self, input):
        val = np.exp(input) / np.sum(np.exp(input))
        return val

    def backward(self, output, learning_rate):
        return output


class FlatteningLayer:
    
    def forward(self, input):
        return input.reshape(input.shape[0], -1)

    def backward(self, output, learning_rate):
        return output.reshape(output.shape[0], -1)


class DenseLayer:

    def __init__(self, n_output):
        self.n_output = n_output
        self.weights = None
        self.biases = None

    def forward(self, input):

        batch_size, n_input = input.shape

        if self.weights is None:
            self.weights = np.random.randn(n_input, self.n_output) / np.sqrt(n_input)
        if self.biases is None:
            self.biases = np.random.randn(self.n_output)

        output = np.dot(input, self.weights) + self.biases
        return output



In [166]:
# batch_size, n_channel, height, width
# input = np.random.randn(*input_shape)
input_shape = (1, 1, 3, 3)
input = np.random.randint(-10, 10, size=input_shape)

# n_filter, filter_size, stride, padding
con = ConvolutionLayer(2, 1, 1, 1)
max = MaxPoolingLayer(pool_size=2, stride=1)
relu = ReLUActivationLayer()
flat = FlatteningLayer()
smax = SoftMaxLayer()
dens = DenseLayer(10)

# output = smax.forward(input)
output = flat.forward(input)
output = dens.forward(output)
# output = relu.forward(input)
# output = max.forward(input)
# output = con.forward(input)


print(input)
# print(output.shape)
print(output)

[[[[  9   1   6]
   [  7   0 -10]
   [  2   2  -2]]]]
[[ 4.82816969  1.86551264  1.4269397  -4.88405721  3.64004782 -2.65252299
   2.8304372  -2.77035312  1.18183446  1.18018694]]
